![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

rentals = pd.read_csv('rental_info.csv', parse_dates=['rental_date','return_date'])
display(rentals.head(), rentals.info(), rentals.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   rental_date       15861 non-null  datetime64[ns, UTC]
 1   return_date       15861 non-null  datetime64[ns, UTC]
 2   amount            15861 non-null  float64            
 3   release_year      15861 non-null  float64            
 4   rental_rate       15861 non-null  float64            
 5   length            15861 non-null  float64            
 6   replacement_cost  15861 non-null  float64            
 7   special_features  15861 non-null  object             
 8   NC-17             15861 non-null  int64              
 9   PG                15861 non-null  int64              
 10  PG-13             15861 non-null  int64              
 11  R                 15861 non-null  int64              
 12  amount_2          15861 non-null  float64            
 13  l

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


None

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
count,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000
mean,4.217161,2006.885379,2.944101,114.994578,20.224727,0.204842,0.200303,0.223378,0.198726,23.355504,14832.841876,11.389287
std,2.360383,2.025027,1.649766,40.114715,6.083784,0.403599,0.400239,0.416523,0.399054,23.503164,9393.431996,10.005293
min,0.990000,2004.000000,0.990000,46.000000,9.990000,0.000000,0.000000,0.000000,0.000000,0.980100,2116.000000,0.980100
25%,2.990000,2005.000000,0.990000,81.000000,14.990000,0.000000,0.000000,0.000000,0.000000,8.940100,6561.000000,0.980100
50%,3.990000,2007.000000,2.990000,114.000000,20.990000,0.000000,0.000000,0.000000,0.000000,15.920100,12996.000000,8.940100
75%,4.990000,2009.000000,4.990000,148.000000,25.990000,0.000000,0.000000,0.000000,0.000000,24.900100,21904.000000,24.900100
max,11.990000,2010.000000,4.990000,185.000000,29.990000,1.000000,1.000000,1.000000,1.000000,143.760100,34225.000000,24.900100


1. Getting the number of rental days.

In [18]:
rentals['rental_days'] = rentals['return_date'] - rentals['rental_date']
rentals['rental_days_len'] = rentals['rental_days'].dt.days

In [19]:
rentals.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_days,rental_days_len
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3 days 20:46:00,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 20:05:00,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7 days 05:44:00,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 02:24:00,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4 days 01:05:00,4


2. Adding dummy variables using the special features column.

In [3]:
pd.set_option('display.max_columns', None)

In [20]:
dummies = pd.get_dummies(rentals['special_features'])

df = pd.concat([rentals, dummies], axis=1).drop(columns=['special_features'])
display(df.head())

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_days,rental_days_len,"{""Behind the Scenes""}","{""Deleted Scenes"",""Behind the Scenes""}","{""Deleted Scenes""}","{Commentaries,""Behind the Scenes""}","{Commentaries,""Deleted Scenes"",""Behind the Scenes""}","{Commentaries,""Deleted Scenes""}",{Commentaries},"{Trailers,""Behind the Scenes""}","{Trailers,""Deleted Scenes"",""Behind the Scenes""}","{Trailers,""Deleted Scenes""}","{Trailers,Commentaries,""Behind the Scenes""}","{Trailers,Commentaries,""Deleted Scenes"",""Behind the Scenes""}","{Trailers,Commentaries,""Deleted Scenes""}","{Trailers,Commentaries}",{Trailers}
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,3 days 20:46:00,3,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,2 days 20:05:00,2,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,7 days 05:44:00,7,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,2 days 02:24:00,2,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,4 days 01:05:00,4,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


3. Executing a train-test split

In [21]:
X = df.drop(columns=['rental_date', 'return_date', 'rental_days','release_year', 'rental_days_len'])
y = df['rental_days_len'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

4. Performing feature selection

In [22]:
# Using Lasso

from sklearn.linear_model import Lasso

# Scaler

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

lasso = Lasso(alpha=0.1, random_state=42, max_iter=10000)

lasso.fit(X_train_scaled, y_train)

lasso_coef = lasso.coef_



In [23]:
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

5. Choosing models and performing hyperparameter tuning

In [27]:
# For tunning 
from sklearn.model_selection import RandomizedSearchCV

In [34]:
# OLS
from sklearn.linear_model import LinearRegression

ols = LinearRegression(n_jobs=-1)

ols.fit(X_lasso_train, y_train)


LinearRegression(n_jobs=-1)

In [42]:
#RandomForest
from sklearn.ensemble import RandomForestRegressor

# Random forest hyperparameter space
param_grid = {'n_estimators': np.arange(1,101,1),
          'max_depth':np.arange(1,11,1)}

rf = RandomForestRegressor(n_jobs=-1)

rf_search = RandomizedSearchCV(rf, 
                                 param_distributions=param_grid, 
                                 cv=5, 
                                 random_state=42)


rf_search.fit(X_train, y_train)

hyper_params = rf_search.best_params_

rf = RandomForestRegressor(n_estimators=hyper_params["n_estimators"], 
                           max_depth=hyper_params["max_depth"], 
                           random_state=42)
rf.fit(X_train,y_train)


RandomForestRegressor(max_depth=9, n_estimators=61, random_state=42)

6. Predicting values on test set

In [43]:
# OLS
ols_pred = ols.predict(X_lasso_test)

# RFR
rf_pred = rf.predict(X_test)

7. Computing mean squared error

In [44]:
# OLS
mse_lin_reg_lasso = mean_squared_error(y_test, ols_pred)
print(f'Linear MSE: {mse_lin_reg_lasso}')

#RFR
mse_random_forest= mean_squared_error(y_test, rf_pred)
print(f'RFR MSE: {mse_random_forest}')

Linear MSE: 4.842493840214282
RFR MSE: 2.2371992145485233


In [45]:
# Best Model and mse

best_model = rf

best_mse = mse_random_forest